In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnbsl import *

from classes import *
from utils import *

In [3]:
SEASON = '21'
set_seed(int(SEASON))

## Download HKJC odds

In [4]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [5]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [6]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [7]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [8]:
matches_json = json.loads(response.text)[1]['matches']

In [9]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-09-21-14-03-17.txt').read())[1]['matches']

In [10]:
matches = [Match(m) for m in matches_json if m['matchStatus'] == 'Defined']
odds = [m.export() for m in matches]

['20211119FRI2', '2021-Nov-20', '00:00:00', 'Russian Premier [RPL]', 'Zenit St. Petersburg', 'FC Nizhny Novgorod', 'false', '12.5', '4.05', '1.19', 'false', '10.5', '2.40', '1.50', 'true', '9.5', '1.90', '1.80']
['20211119FRI3', '2021-Nov-20', '01:30:00', 'German Division 2 [GD2]', 'Hannover', 'Paderborn', 'false', '13.5', '4.75', '1.14', 'false', '10.5', '2.10', '1.65', 'true', '9.5', '1.73', '1.98']
['20211119FRI4', '2021-Nov-20', '01:30:00', 'German Division 2 [GD2]', 'Sandhausen', 'Nurnberg', 'true', '9.5', '1.73', '1.98', 'false', '13.5', '4.75', '1.14', 'false', '10.5', '2.10', '1.65']
['20211119FRI15', '2021-Nov-20', '03:30:00', 'German Division 1 [GSL]', 'Augsburg', 'Bayern Munich', 'false', '11.5', '2.70', '1.40', 'true', '10.5', '2.15', '1.62', 'false', '13.5', '4.55', '1.15']
['20211119FRI17', '2021-Nov-20', '03:45:00', 'Eng Championship [ED1]', 'QPR', 'Luton', 'true', '10.5', '1.95', '1.75', 'false', '11.5', '2.48', '1.47', 'false', '13.5', '4.05', '1.19']
['20211119FRI18',

In [11]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['alpha_1', 'alpha_2', 'mu_1', 'mu_2', 'omega', 'corr']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [12]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [13]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [14]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [15]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [16]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [17]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [18]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [19]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [20]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [21]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [22]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [23]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [24]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [25]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [26]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [27]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [28]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [29]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [30]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [31]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,FRI3,FRI4,SAT19,SAT20,SAT21
Date,2021-11-20 00:00:00,2021-11-20 00:00:00,2021-11-20 00:00:00,2021-11-20 00:00:00,2021-11-20 00:00:00
Time,01:30:00,01:30:00,20:30:00,20:30:00,20:30:00
LeagueJC,German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2]
HomeTeamJC,Hannover,Sandhausen,Darmstadt,Hamburg,Rostock
AwayTeamJC,Paderborn,Nurnberg,St. Pauli,Jahn Regensburg,Aue
MAINLINE_0,false,true,true,false,false
CHL_LINE_0,13.5,9.5,9.5,13.5,12.5
CHL_H_0,4.75,1.73,1.77,4.3,4.55
CHL_L_0,1.14,1.98,1.93,1.17,1.15


## Load model

In [32]:
learn_bnb = load_learner('models/learn_bnbsl.pkl')

In [33]:
to = load_pandas('models/to.pkl')

In [34]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [35]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [36]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,Dayofweek,FTHGLast5Avg,HSLast5Avg,HSTLast5Avg,HCLast5Avg,FTAGLast5Avg,ASLast5Avg,ASTLast5Avg,ACLast5Avg,HomeFTGForLast5Avg,HomeSForLast5Avg,HomeSTForLast5Avg,HomeCForLast5Avg,AwayFTGForLast5Avg,AwaySForLast5Avg,AwaySTForLast5Avg,AwayCForLast5Avg,HomeFTGAgainstLast5Avg,HomeSAgainstLast5Avg,HomeSTAgainstLast5Avg,HomeCAgainstLast5Avg,AwayFTGAgainstLast5Avg,AwaySAgainstLast5Avg,AwaySTAgainstLast5Avg,AwayCAgainstLast5Avg,Year,Month,Week,Day,Dayofyear
0,D2,Hannover,Paderborn,4,0.6,11.2,4.2,5.4,2.8,13.8,5.8,3.0,0.6,11.6,3.4,5.0,2.0,14.8,7.0,5.0,1.4,16.2,4.4,5.6,1.8,20.4,8.0,6.6,2021.0,11.0,46.0,19.0,323.0
1,D2,Sandhausen,Nurnberg,4,0.4,9.2,2.8,5.0,1.4,11.2,4.2,4.8,1.4,8.2,3.2,2.4,1.4,12.0,3.6,4.6,2.6,17.4,6.6,5.6,0.8,13.0,3.4,6.4,2021.0,11.0,46.0,19.0,323.0
2,D2,Darmstadt,St Pauli,5,2.8,13.2,5.2,3.2,1.6,11.4,4.6,4.2,2.6,13.8,5.6,4.2,3.6,16.0,8.0,5.6,0.8,14.2,4.0,5.0,0.8,9.4,2.2,2.6,2021.0,11.0,46.0,20.0,324.0
3,D2,Hamburg,Regensburg,5,1.6,19.6,7.0,10.0,1.4,11.4,4.0,3.2,1.4,17.0,5.4,8.8,2.4,13.4,5.6,3.6,1.2,10.2,4.6,5.2,1.2,14.4,4.6,4.8,2021.0,11.0,46.0,20.0,324.0
4,D2,Hansa Rostock,Erzgebirge Aue,5,1.0,13.4,4.0,6.6,0.8,12.0,3.8,3.0,1.0,10.8,3.4,3.4,1.2,14.8,4.2,5.0,1.6,10.8,5.4,4.4,1.4,11.4,3.8,4.8,2021.0,11.0,46.0,20.0,324.0
5,D2,Werder Bremen,Schalke 04,5,1.6,15.6,5.0,7.2,1.6,15.6,5.8,4.4,1.2,14.0,3.8,6.8,1.8,14.2,6.0,4.4,1.8,11.6,4.0,2.2,0.2,11.8,2.4,5.6,2021.0,11.0,46.0,20.0,324.0
6,D2,Heidenheim,Holstein Kiel,6,1.4,15.8,4.2,9.0,1.4,12.6,3.8,4.8,1.0,12.8,3.6,6.8,1.0,14.2,4.8,7.0,2.4,11.8,5.2,6.4,1.2,12.8,4.0,4.0,2021.0,11.0,46.0,21.0,325.0
7,D2,Dresden,Fortuna Dusseldorf,6,1.6,14.2,6.4,4.6,1.0,13.6,3.2,6.4,0.6,13.6,4.0,5.4,1.8,14.2,5.2,6.0,1.6,12.6,4.8,5.2,1.6,15.0,5.2,4.6,2021.0,11.0,46.0,21.0,325.0
8,D2,Ingolstadt,Karlsruhe,6,0.6,13.8,3.0,4.8,1.6,11.8,4.2,5.4,0.4,11.2,2.2,3.0,1.6,14.2,4.4,4.2,2.0,11.2,3.6,5.4,2.6,15.2,6.2,4.0,2021.0,11.0,46.0,21.0,325.0
9,D1,Augsburg,Bayern Munich,4,1.0,11.4,3.6,5.2,3.0,18.6,7.4,6.4,1.4,10.6,5.0,4.8,3.6,17.8,7.8,5.0,2.2,15.4,5.2,5.8,1.2,8.2,4.2,3.8,2021.0,11.0,46.0,19.0,323.0


In [37]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [38]:
alpha = F.softplus(pred[:, 0:2])
mu = F.softplus(pred[:, 2:4])
omega = pred[:, 4]

total_count = 1. / alpha
logits = torch.log(alpha * mu)

# print(alpha, mu, omega)

In [39]:
# Method 1:
corr = omega * torch.sqrt(mu.prod(-1)) * alpha.prod(-1) / ((1.+alpha)**(1./alpha+1.)).prod(-1) / torch.sqrt((1.+alpha*mu).prod(-1))

In [40]:
# Method 2:
# d = 1.-math.exp(-1)
# corr = omega * d**2 * torch.sqrt(mu.prod(-1)*(1.+alpha*mu).prod(-1)) * ((1.+d*alpha*mu)**(-1-1/alpha)).prod(-1)

In [41]:
odds[cols_pred] = torch.cat([alpha, mu, omega.unsqueeze(-1), corr.unsqueeze(-1)], dim=-1)

In [42]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [43]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [44]:
odds.head(10)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,alpha_1,alpha_2,mu_1,mu_2,omega,corr
0,FRI3,2021-11-20,01:30:00,German Division 2 [GD2],Hannover,Paderborn,False,13.5,4.75,1.14,0.048484,0.048776,5.418106,5.438321,-26.240854,-0.034382
1,FRI4,2021-11-20,01:30:00,German Division 2 [GD2],Sandhausen,Nurnberg,True,9.5,1.73,1.98,0.035783,0.028046,4.731163,5.404799,-2.048508,-0.001175
2,SAT19,2021-11-20,20:30:00,German Division 2 [GD2],Darmstadt,St. Pauli,True,9.5,1.77,1.93,0.080363,0.072458,4.625913,5.214601,-30.848240,-0.080610
3,SAT20,2021-11-20,20:30:00,German Division 2 [GD2],Hamburg,Jahn Regensburg,False,13.5,4.30,1.17,0.028503,0.056931,6.715641,4.183147,11.761248,0.010807
4,SAT21,2021-11-20,20:30:00,German Division 2 [GD2],Rostock,Aue,False,12.5,4.55,1.15,0.083263,0.150980,5.552598,3.082833,-22.201670,-0.095397
5,SAT89,2021-11-21,03:30:00,German Division 2 [GD2],Werder Bremen,Schalke 04,True,9.5,1.85,1.85,0.103552,0.096058,4.633359,4.895111,-35.452320,-0.139900
6,SUN22,2021-11-21,20:30:00,German Division 2 [GD2],Heidenheim,Holstein Kiel,False,11.5,2.40,1.50,0.039981,0.047419,6.043583,5.670304,-31.484612,-0.036087
7,SUN23,2021-11-21,20:30:00,German Division 2 [GD2],Dresden,Dusseldorf,True,9.5,1.75,1.95,0.080524,0.104138,4.964637,4.110312,-38.949306,-0.129130
8,SUN24,2021-11-21,20:30:00,German Division 2 [GD2],Ingolstadt,Karlsruher,True,9.5,1.95,1.75,0.075569,0.059662,4.960307,5.305119,-32.188572,-0.070097
9,FRI15,2021-11-20,03:30:00,German Division 1 [GSL],Augsburg,Bayern Munich,False,11.5,2.70,1.40,0.312597,0.075350,1.582664,6.826084,-34.745243,-0.202327


In [45]:
prob_hilo = []

for r in list(zip(odds['alpha_1'], odds['alpha_2'], odds['mu_1'], odds['mu_2'], odds['omega'], odds['CHL_LINE'])):
    total_count = 1. / torch.tensor(r[0:2], device='cpu') 
    logits = torch.log(torch.tensor(r[0:2], device='cpu') * torch.tensor(r[2:4], device='cpu'))
    omega = torch.tensor(r[4], device='cpu')

    bnb_corner = BivariateNegativeBinomialSL(total_count=total_count, omega=omega, logits=logits)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[5]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

/Users/twtang/miniconda3/envs/fastai2/lib/python3.8/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1634272482218/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


In [46]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [47]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [48]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [49]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [50]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [51]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [52]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [53]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
4,FRI15,2021-11-20,03:30:00,German Division 1 [GSL],Augsburg,Bayern Munich,True,10.5,2.15,1.62,0.246267,0.753733,0.356527,Low,$$$
6,SUN41,2021-11-21,23:15:00,Spanish Division 1 [SFL],CF Granada,Real Madrid,True,9.5,1.92,1.78,0.294522,0.705478,0.327887,Low,$$$
9,SUN30,2021-11-21,22:00:00,Eng Premier [EPL],Manchester City,Everton,True,11.5,2.15,1.62,0.267477,0.732523,0.301108,Low,$$$
11,SAT62,2021-11-20,23:15:00,Spanish Division 1 [SFL],Sevilla,Alaves,True,8.5,1.73,1.98,0.702389,0.297611,0.294702,High,$$
12,SAT17,2021-11-20,20:30:00,Eng Premier [EPL],Leicester,Chelsea,True,10.5,1.80,1.90,0.347615,0.652385,0.266146,Low,$$
17,SAT74,2021-11-21,01:30:00,German Division 1 [GSL],Union Berlin,Hertha Berlin,True,9.5,2.05,1.68,0.325554,0.674446,0.195690,Low,$
19,MON10,2021-11-23,03:45:00,Italian Division 1 [ISA],AC Torino,Udinese,True,9.5,2.00,1.72,0.595112,0.404888,0.190223,High,$
20,SUN22,2021-11-21,20:30:00,German Division 2 [GD2],Heidenheim,Holstein Kiel,True,10.5,1.95,1.75,0.604353,0.395647,0.187883,High,$
23,FRI19,2021-11-20,04:00:00,French Division 1 [FFL],Monaco,Lille,True,9.5,2.00,1.72,0.346526,0.653474,0.172188,Low,$
26,FRI17,2021-11-20,03:45:00,Eng Championship [ED1],QPR,Luton,True,10.5,1.95,1.75,0.360805,0.639195,0.158121,Low,$


In [54]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,FRI15,2021-11-20,03:30:00,German Division 1 [GSL],Augsburg,Bayern Munich,False,13.5,4.55,1.15,0.070895,0.929105,0.456473,Low,None
1,SUN41,2021-11-21,23:15:00,Spanish Division 1 [SFL],CF Granada,Real Madrid,False,13.5,5.55,1.10,0.052915,0.947085,0.417939,Low,None
2,FRI15,2021-11-20,03:30:00,German Division 1 [GSL],Augsburg,Bayern Munich,False,11.5,2.70,1.40,0.169462,0.830538,0.406883,Low,None
3,SUN41,2021-11-21,23:15:00,Spanish Division 1 [SFL],CF Granada,Real Madrid,False,10.5,2.48,1.47,0.204891,0.795109,0.359172,Low,None
4,FRI15,2021-11-20,03:30:00,German Division 1 [GSL],Augsburg,Bayern Munich,True,10.5,2.15,1.62,0.246267,0.753733,0.356527,Low,$$$
5,SAT17,2021-11-20,20:30:00,Eng Premier [EPL],Leicester,Chelsea,False,14.5,4.75,1.14,0.079924,0.920076,0.349193,Low,None
6,SUN41,2021-11-21,23:15:00,Spanish Division 1 [SFL],CF Granada,Real Madrid,True,9.5,1.92,1.78,0.294522,0.705478,0.327887,Low,$$$
7,SUN30,2021-11-21,22:00:00,Eng Premier [EPL],Manchester City,Everton,False,14.5,4.40,1.16,0.092861,0.907139,0.326755,Low,None
8,SUN30,2021-11-21,22:00:00,Eng Premier [EPL],Manchester City,Everton,False,12.5,2.75,1.39,0.194083,0.805917,0.308268,Low,None
9,SUN30,2021-11-21,22:00:00,Eng Premier [EPL],Manchester City,Everton,True,11.5,2.15,1.62,0.267477,0.732523,0.301108,Low,$$$


In [55]:
odds.to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)

## END